<a href="https://colab.research.google.com/github/Julian6262/the_founder/blob/main/home%20work%2014/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai mwclient mwparserfromhell tiktoken aiogram

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import mwclient, mwparserfromhell
import openai, tiktoken
import os, getpass
import pandas as pd
import re
from scipy import spatial
import ast

In [ ]:
CATEGORY_TITLE = "Category:Russian chemists"
WIKI_SITE = "en.wikipedia.org"

GPT_MODEL = "gpt-3.5-turbo"
EMBEDDING_MODEL = "text-embedding-ada-002"

SAVE_PATH = "./Russian_chemists.csv"

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")
client = openai.OpenAI(api_key = os.environ.get("OPENAI_API_KEY"))

Введите OpenAI API Key:··········


### **Создаем базу знаний по запросу "Russian_chemists"**

**Сылка на готовый файл базы знаний:**  https://drive.google.com/file/d/1A_q0v0mu8JI4YZYUbVlb2qcmSfGqxL6i/view?usp=sharing

In [ ]:
# Соберем заголовки всех статей
def titles_from_category(category: mwclient.listing.Category, max_depth: int) -> set[str]:
    """Возвращает набор заголовков страниц в данной категории Википедии и ее подкатегориях."""
    titles = set() # Используем множество для хранения заголовков статей
    for cm in category.members(): # Перебираем вложенные объекты категории
        if type(cm) == mwclient.page.Page: # Если объект является страницей
            titles.add(cm.name) # в хранилище заголовков добавляем имя страницы
        elif isinstance(cm, mwclient.listing.Category) and max_depth > 0: # Если объект является категорией и глубина вложения не достигла максимальной
            deeper_titles = titles_from_category(cm, max_depth=max_depth - 1) # вызываем рекурсивно функцию для подкатегории
            titles.update(deeper_titles) # добавление в множество элементов из другого множества
    return titles

site = mwclient.Site(WIKI_SITE)

# Загрузка раздела заданной категории
category_page = site.pages[CATEGORY_TITLE]
# Получение множества всех заголовков категории с вложенностью на один уровень
titles = titles_from_category(category_page, max_depth=1)


print(f"Создано {len(titles)} заголовков статей в категории {CATEGORY_TITLE}.")

Создано 192 заголовков статей в категории Category:Russian_chemists.


In [ ]:
# Задаем секции, которые будут отброшены при парсинге статей
SECTIONS_TO_IGNORE = [
    "See also",
    "References",
    "External links",
    "Further reading",
    "Footnotes",
    "Bibliography",
    "Sources",
    "Citations",
    "Literature",
    "Footnotes",
    "Notes and references",
    "Photo gallery",
    "Works cited",
    "Photos",
    "Gallery",
    "Notes",
    "References and sources",
    "References and notes",
]

In [ ]:
# Функция возвращает список всех вложенных секций для заданной секции страницы Википедии
def all_subsections_from_section(section: mwparserfromhell.wikicode.Wikicode, parent_titles: list[str], sections_to_ignore: set[str],) -> list[tuple[list[str], str]]:
    # Извлекаем заголовки текущей секции
    headings = [str(h) for h in section.filter_headings()]
    title = headings[0]
    # Заголовки Википедии имеют вид: "== Heading =="

    if title.strip("=" + " ") in sections_to_ignore:
        # Если заголовок секции в списке для игнора, то пропускаем его
        return []

    # Объединим заголовки и подзаголовки, чтобы сохранить контекст для chatGPT
    titles = parent_titles + [title]

    # Преобразуем wikicode секции в строку
    full_text = str(section)

    # Выделяем текст секции без заголовка
    section_text = full_text.split(title)[1]
    if len(headings) == 1:
        # Если один заголовок, то формируем результирующий список
        return [(titles, section_text)]
    else:
        first_subtitle = headings[1]
        section_text = section_text.split(first_subtitle)[0]
        # Формируем результирующий список из текста до первого подзаголовка
        results = [(titles, section_text)]
        for subsection in section.get_sections(levels=[len(titles) + 1]):
            results.extend(
                # Вызываем функцию получения вложенных секций для заданной секции
                all_subsections_from_section(subsection, titles, sections_to_ignore)
                )  # Объединяем результирующие списки данной функции и вызываемой
        return results


# Функция возвращает список всех секций страницы, за исключением тех, которые отбрасываем
def all_subsections_from_title(title: str, sections_to_ignore: set[str] = SECTIONS_TO_IGNORE, site_name: str = WIKI_SITE,) -> list[tuple[list[str], str]]:

    site = mwclient.Site(site_name)

    # Запрашиваем страницу по заголовку
    page = site.pages[title]

    # Получаем текстовое представление страницы
    text = page.text()

    # Удобный парсер для MediaWiki
    parsed_text = mwparserfromhell.parse(text)
    # Извлекаем заголовки
    headings = [str(h) for h in parsed_text.filter_headings()]
    if headings: # Если заголовки найдены
        # В качестве резюме берем текст до первого заголовка
        summary_text = str(parsed_text).split(headings[0])[0]
    else:
        # Если нет заголовков, то весь текст считаем резюме
        summary_text = str(parsed_text)
    results = [([title], summary_text)] # Добавляем резюме в результирующий список
    for subsection in parsed_text.get_sections(levels=[2]): # Извлекаем секции 2-го уровня
        results.extend(
            # Вызываем функцию получения вложенных секций для заданной секции
            all_subsections_from_section(subsection, [title], sections_to_ignore)
        ) # Объединяем результирующие списки данной функции и вызываемой
    return results

In [ ]:
wikipedia_sections = []
for title in titles:
    wikipedia_sections.extend(all_subsections_from_title(title))
print(f"Найдено {len(wikipedia_sections)} секций на {len(titles)} страницах")

Найдено 771 секций на 192 страницах


In [ ]:
# Очистка текста секции от ссылок <ref>xyz</ref>, начальных и конечных пробелов
def clean_section(section: tuple[list[str], str]) -> tuple[list[str], str]:
    titles, text = section
    # Удаляем ссылки
    text = re.sub(r"<ref.*?</ref>", "", text)
    # Удаляем пробелы вначале и конце
    text = text.strip()
    return (titles, text)


def keep_section(section: tuple[list[str], str]) -> bool:
    """Возвращает значение True, если раздел должен быть сохранен, в противном случае значение False."""
    titles, text = section
    # Фильтруем по произвольной длине, можно выбрать и другое значение
    return False if len(text) < 16 else True


wikipedia_sections = [clean_section(ws) for ws in wikipedia_sections]
original_num_sections = len(wikipedia_sections)
wikipedia_sections = [ws for ws in wikipedia_sections if keep_section(ws)]
print(f"Отфильтровано {original_num_sections-len(wikipedia_sections)} секций, осталось {len(wikipedia_sections)} секций.")

Отфильтровано 23 секций, осталось 748 секций.


In [ ]:
# Функция подсчета токенов
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


# Функция разделения строк
def halved_by_delimiter(string: str, delimiter: str = "\n") -> list[str, str]:
    """Разделяет строку надвое с помощью разделителя (delimiter), пытаясь сбалансировать токены с каждой стороны."""

    # Делим строку на части по разделителю, по умолчанию \n - перенос строки
    chunks = string.split(delimiter)
    if len(chunks) == 1:
        return [string, ""]  # разделитель не найден
    elif len(chunks) == 2:
        return chunks  # нет необходимости искать промежуточную точку
    else:
        # Считаем токены
        total_tokens = num_tokens(string)
        halfway = total_tokens // 2
        # Предварительное разделение по середине числа токенов
        best_diff = halfway
        # В цикле ищем какой из разделителей, будет ближе всего к best_diff
        for i, chunk in enumerate(chunks):
            left = delimiter.join(chunks[: i + 1])
            left_tokens = num_tokens(left)
            diff = abs(halfway - left_tokens)
            if diff >= best_diff:
                break
            else:
                best_diff = diff
        left = delimiter.join(chunks[:i])
        right = delimiter.join(chunks[i:])
        # Возвращаем левую и правую часть оптимально разделенной строки
        return [left, right]


# Функция обрезает строку до максимально разрешенного числа токенов
def truncated_string(string: str, model: str, max_tokens: int, print_warning: bool = True,) -> str:
    """Обрезка строки до максимально разрешенного числа токенов."""
    encoding = tiktoken.encoding_for_model(model)
    encoded_string = encoding.encode(string)
    # Обрезаем строку и декодируем обратно
    truncated_string = encoding.decode(encoded_string[:max_tokens])
    if print_warning and len(encoded_string) > max_tokens:
        print(f"Предупреждение: Строка обрезана с {len(encoded_string)} токенов до {max_tokens} токенов.")
    # Усеченная строка
    return truncated_string


# Функция делит секции статьи на части по максимальному числу токенов
def split_strings_from_subsection(subsection: tuple[list[str], str], max_tokens: int = 1000, model: str = GPT_MODEL, max_recursion: int = 5,) -> list[str]:
    """
    Разделяет секции на список из частей секций, в каждой части не более max_tokens.
    Каждая часть представляет собой кортеж родительских заголовков [H1, H2, ...] и текста (str).
    """
    titles, text = subsection
    string = "\n\n".join(titles + [text])
    num_tokens_in_string = num_tokens(string)
    # Если длина соответствует допустимой, то вернет строку
    if num_tokens_in_string <= max_tokens:
        return [string]
    # если в результате рекурсия не удалось разделить строку, то просто усечем ее по числу токенов
    elif max_recursion == 0:
        return [truncated_string(string, model=model, max_tokens=max_tokens)]
    # иначе разделим пополам и выполним рекурсию
    else:
        titles, text = subsection
        for delimiter in ["\n\n", "\n", ". "]: # Пробуем использовать разделители от большего к меньшему (разрыв, абзац, точка)
            left, right = halved_by_delimiter(text, delimiter=delimiter)
            if left == "" or right == "":
                # если какая-либо половина пуста, повторяем попытку с более простым разделителем
                continue
            else:
                # применим рекурсию на каждой половине
                results = []
                for half in [left, right]:
                    half_subsection = (titles, half)
                    half_strings = split_strings_from_subsection(
                        half_subsection,
                        max_tokens=max_tokens,
                        model=model,
                        max_recursion=max_recursion - 1, # уменьшаем максимальное число рекурсий
                    )
                    results.extend(half_strings)
                return results
    # иначе никакого разделения найдено не было, поэтому просто обрезаем строку (должно быть очень редко)
    return [truncated_string(string, model=model, max_tokens=max_tokens)]

In [ ]:
# Делим секции на части
MAX_TOKENS = 1600
wikipedia_strings = []
for section in wikipedia_sections:
    wikipedia_strings.extend(split_strings_from_subsection(section, max_tokens=MAX_TOKENS))

print(f"{len(wikipedia_sections)} секций Википедии поделены на {len(wikipedia_strings)} строк.")

748 секций Википедии поделены на 775 строк.


-- Код ниже не выполняем, есть готовый файл --

In [ ]:
# Функция отправки chatGPT строки для ее токенизации (вычисления эмбедингов)
# def get_embedding(text, model="text-embedding-ada-002"):
#    return client.embeddings.create(input = [text], model=model).data[0].embedding


# df = pd.DataFrame({"text": wikipedia_strings})

# df['embedding'] = df.text.apply(lambda x: get_embedding(x, model='text-embedding-ada-002'))

In [ ]:
# df.head()

In [ ]:
# Сохранение результата
# df.to_csv(SAVE_PATH, index=False)

### **Метод дообучения chatGPT Search-Ask**


In [ ]:
df = pd.read_csv(SAVE_PATH)
df['embedding'] = df['embedding'].apply(ast.literal_eval)

In [ ]:
def strings_ranked_by_relatedness(query: str, df: pd.DataFrame, relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y), top_n: int = 100) -> tuple[list[str], list[float]]:
    """Возвращает строки и схожести, отсортированные от большего к меньшему"""
    # Отправляем в OpenAI API пользовательский запрос для токенизации
    query_embedding_response = openai.embeddings.create(model=EMBEDDING_MODEL,input=query,)
    # Получен токенизированный пользовательский запрос
    query_embedding = query_embedding_response.data[0].embedding
    # Сравниваем пользовательский запрос с каждой токенизированной строкой DataFrame
    strings_and_relatednesses = [(row["text"], relatedness_fn(query_embedding, row["embedding"])) for i, row in df.iterrows()]
    # Сортируем по убыванию схожести полученный список
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    # Преобразовываем наш список в кортеж из списков
    strings, relatednesses = zip(*strings_and_relatednesses)
    # Возвращаем n лучших результатов
    return strings[:top_n], relatednesses[:top_n]

In [ ]:
# Проверка на релевантность по запросу
strings, relatednesses = strings_ranked_by_relatedness("Olga Bogdanova (chemist)", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.941


'Olga Bogdanova (chemist)\n\n== Biography ==\n\nIn the 1920s Bogdanova worked in a laboratory at a [[synthetic rubber]] factory.\n\nFrom the early 1930s she worked at the [http://www.ioc.ac.ru/english N. D. Zelinsky Institute of Organic Chemistry] of the [[Academy of Sciences of the Soviet Union]] (now: [[Russian Academy of Sciences]]). Bordanova was a student and, for many years, a colleague of [[Academician]]s [[Nikolay Zelinsky|N. D. Zelinsky]] and [[Aleksei Balandin|A. A. Balandin]].'

relatedness=0.935


"Olga Bogdanova (chemist)\n\n{{short description|Soviet chemist}}\n\n{{Infobox scientist\n| name              = Olga Bogdanova\n| image             = Bogdanova O.K.jpg\n| birth_place       = [[Mogilev]], [[Russian Empire]]\n| caption           = Bogdanova in 1950\n| birth_name        = Olga Konstantinovna Bogdanova\n| birth_date        = 29 June n.s. (11 July o.s.) 1896\n| death_date        = March 1982 (aged 85)\n| citizenship       = [[USSR]]\n| workplaces        = [[Academy of Sciences of the Soviet Union]], Institute of Organic Chemistry\n| awards            = [[Order of the Red Banner of Labour]] 1953, 1967;\nMedal “For Valorous Labour during the Great Patriotic War, 1941-1945”;\t\n<br>Medal “In Honor of the 800th Anniversary of the Founding of Moscow”;\n[[USSR State Prize|Stalin Prize]] 1950\n}}\n\n'''Olga Konstantinovna Bogdanova''' ([[Russian language|Russian]]: Ольга Константиновна Богда́нова; 29 June [[Old Style and New Style dates|n.s.]], 1896 — March 1982)\xa0was a [[Soviet

relatedness=0.933


'Olga Bogdanova (chemist)\n\n== Awards and prizes ==\n\n* [[USSR State Prize|Stalin Prize]], 2nd degree (1950) – for the development and industrial application of a catalyst used in a new chemical process\n* [[Order of the Red Banner of Labour]] (1953, 1967)\n* Medals<ref name=":0" />'

relatedness=0.906


'Olga Bogdanova (chemist)\n\n== Biography ==\n\n=== Main achievements ===\n\n* In 1941—1942 A. A. Balandin, O. K. Bogdanova, and A. P. Shcheglova developed and implemented at a synthetic rubber factory a method for producing a gas-resistant (polysulfide or thio) rubber, which was widely used in the production of a “self-tightening” – or, more accurately, “self-sealing” when struck by bullets – coating for aircraft fuel tanks.\n* In 1946—1952, a new method was developed for obtaining 1,3 butadiene from petroleum feedstock on chromium oxide catalysts, which found industrial application at synthetic rubber factories in Sterlitamak and Sumgait (O. K. Bogdanova and\xa0A. P. Shcheglova)\n* In 1974—1981 O. K. Bogdanova and D. P. Belomestnykh developed a way to produce styrene (vinylbenzene) and its homologues through the oxidative decomposition of alkyl-benzene over a complex chromium oxide catalyst, which surpassed all known industrial catalysts used to decompose ethylbenzene.\n\nBogdanova h

relatedness=0.891


'Olga Dontsova\n\n==Education and work==\n\nIn 1991 she defended her Ph.D. from the Chemistry Department of [[Moscow State University]] and in 1997 her doctoral dissertation \'\'development of chemical methods for studying the structure and function of complex ribonucleoprotein systems\'\' was published. Since 1999 she is the professor of Bio-organic chemistry, Department of Chemistry of Natural Compounds at [[MSU Faculty of Chemistry|Faculty of Chemistry]], [[Moscow State University]].<ref name="b"/>\n\nShe is the member of the [[Russian Foundation for Basic Research|Russian Foundation for Basic Research Board]].'

In [ ]:
name_category = CATEGORY_TITLE.split(":")[1]


def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Возвращает число токенов в строке для заданной модели"""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

# Функция формирования запроса к chatGPT по пользовательскому вопросу и базе знаний
def query_message(query: str, df: pd.DataFrame, model: str, token_budget: int) -> str:
    """Возвращает сообщение для GPT с соответствующими исходными текстами, извлеченными из фрейма данных (базы знаний)."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df) # получаем строки из базы знаний и соответствующую релевантность в сравнении этой строки и заданного вопроса
    # Шаблон инструкции для chatGPT
    message = f'Use the below articles on the {name_category} to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    # Шаблон для вопроса
    question = f"\n\nQuestion: {query}"

    # Добавляем к сообщению для chatGPT релевантные строки из базы знаний, пока не выйдем за допустимое число токенов
    for string in strings:
        next_article = f'\n\nWikipedia article section:\n"""\n{string}\n"""'
        if (num_tokens(message + next_article + question, model=model) > token_budget):
            break
        else:
            message += next_article
    return message + question


def ask(query: str, df: pd.DataFrame = df, model: str = GPT_MODEL, token_budget: int = 4096 - 500, print_message: bool = False,) -> str:
    """Отвечает на вопрос, используя GPT и базу знаний."""
    # Формируем сообщение к chatGPT (функция выше)
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)

    messages = [
        {"role": "system", "content": f'You answer questions about the {name_category}.'},
        {"role": "user", "content": message},
    ]
    response = openai.chat.completions.create(model=model, messages=messages,temperature=0)
    response_message = response.choices[0].message.content
    return response_message

In [ ]:
ask('When Valeriy Alekseevich was born?')

'Valery Alekseyevich Legasov was born on September 1, 1936.'

In [ ]:
ask('Scientific career of Yuri Struchkov')

'Yuri Struchkov graduated from the MSU Faculty of Chemistry in 1948. He worked on his Ph.D. thesis under the supervision of professor A.I. Kitaigorodski from 1948 to 1953. In 1950, he joined the staff of the Zelinsky Institute of Organic Chemistry. He defended his PhD thesis on X-Ray structural study in 1953. From 1954 to 1995, he worked at the Nesmeyanov Institute of Organoelement Compounds, initially in the laboratory headed by A.I. Kitaigorodski, then in the diffraction studies group, and later in the new Laboratory of the X-Ray Structure Analysis which he founded and headed. In 1989, Struchkov became the Director of the X-Ray Structural Centre of RAS, based on the INEOS LXRSA. He was elected Corresponding Member of RAS in 1990 and became the vice-president of the International Union of Crystallography from 1993 to 1995.'

In [ ]:
ask('what Russian chemists do you know?')

'I know about several Russian chemists, including Pyotr Kapitsa, Salambek Khadzhiyev, Morris Kharasch, Gottlieb Kirchhoff, Ivan Knunyants, Anatoly V. Oleynik, Boris Jacobi, Nikolay Semyonov, Carl Schmidt, Vladimir Shukhov, Mikhail Shultz, Sergei Vasilyevich Lebedev, Mikhail Lomonosov, Aleksandr Loran, Dmitry Chernov, Aleksei Chichibabin, Lev Chugaev, Karl Ernst Claus, Valeriy Chernyshev, Alexander Baykov, Ernest Beaux, Nikolay Beketov, Friedrich Konrad Beilstein, Boris Pavlovich Belousov, Alexander Borodin, Aleksandr Butlerov, Sergey Reformatsky, Sergey Namyotkin, Konstantin Novoselov, Vladimir Markovnikov, Yurii Sh. Matros, Dmitri Mendeleyev, Nikolai Menshutkin, Nikolay Trifonov, Mikhail Tsvet, Andre Geim, Igor Gorynin, Vladimir Ipatieff, Isidore, Paul Walden, Alexander Arbuzov, Nikolay Demyanov, Aleksandr Dianin, Martin Kabachnik, Dmitrii Knorre, George Koval, Ulmas Mirsaidovich Mirsaidov, Valentin Koptyug, Vasily Korshak, Valery Legasov, and Alexander Zaitsev.'

### **Телеграм Бот**

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import asyncio
from aiogram import Bot, Dispatcher, types, F
from aiogram.filters import Command
from aiogram.types import KeyboardButton
from aiogram.utils.keyboard import ReplyKeyboardBuilder

In [ ]:
os.environ["API_TOKEN"] = getpass.getpass("Введите телерам API TOKEN:")

bot = Bot(token=os.environ.get("API_TOKEN"))
dp = Dispatcher()

Введите телерам API TOKEN:··········


In [ ]:
# Клавиатуры
start_keyboard = ReplyKeyboardBuilder()
start_keyboard.add(KeyboardButton(text='Начать разговор'),
                   KeyboardButton(text='Информация о базе знаний'),
                   )
start_keyboard = start_keyboard.adjust(1, 1).as_markup(resize_keyboard=True, )

In [ ]:
# хэндлеры
@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    await message.answer("Добро пожаловать в GPT бот!", reply_markup=start_keyboard)


@dp.message(Command("help"))
@dp.message(F.text == "Информация о базе знаний")
async def cmd_help(message: types.Message):
    await message.answer(f"Тематика разговора: {name_category}")
    await message.answer(f"Число записей в базе знаний: {len(wikipedia_strings)}")
    await message.answer("Пример запроса к базе: When Valeriy Alekseevich was born?, Evgenii Przhevalsky Biography, who was born on May 20, 1939?, Russian chemists have listed")


@dp.message(F.text == "Начать разговор")
async def cmd_start_text(message: types.Message):
    await message.answer("Введите ваш вопрос то теме:")


@dp.message()
async def cmd_text(message: types.Message):
    reply = ask(message.text)
    await message.answer(f'Ответ: {reply}')


In [ ]:
# Запуск бота
async def main():
    await dp.start_polling(bot)


if __name__ == "__main__":
    asyncio.run(main())